## HW2-0228


ddl：本周四（3月3日）中午12点

有问题请联系微信群联系姜腾

## 1 基础题

#### 1.1 分割范围（考察列表生成式，字符串分割）
输入数字范围的字符串，中间用-连接，以逗号隔开，将字符串表示的范围转化为数字的**生成器**。

举例：
```python
>>> parse_ranges("1-2,4-5,8-10")
<generator object parse_ranges.<locals>.<genexpr> at 0x7ffe7bd0a4d0>
>>> list(parse_ranges("1-2,4-5,8-10"))
[1, 2, 4, 5, 8, 9, 10]
```

In [12]:
def parse_ranges(ranges_string=""):
    # todo
    rang = ranges_string.split(',')
    rang2 = [x.split('-') for x in rang]
    rang3 = (y for x in rang2 for y in range(int(x[0]), int(x[1])+1))
    return rang3
    pass

print(parse_ranges("1-2,4-5,8-10"))
print(list(parse_ranges("1-2,4-5,8-10")))

<generator object parse_ranges.<locals>.<genexpr> at 0x000001359B0B79E0>
[1, 2, 4, 5, 8, 9, 10]


#### 1.2 改进的cache装饰器（考察装饰器和函数可变参数列表）

课上实现了有记忆存储的cache装饰器，但是对输入的参数有限制。现在希望实现一个@cached装饰器，支持对\*args和\*kwargs的输入同样的缓存支持。

注：无需考虑输入不可哈希的情况（学有余力者可以在输入不可哈希时抛出异常，不算分）

In [26]:
import time

def cached(func):
    # todo
    memory = {}
    def inner(farg, *args, **kwargs):
        tup = (farg, args, *kwargs)
        if tup not in memory:
            memory[tup] = func(farg, *args, **kwargs)
        return memory[tup]
    return inner
    pass
    
@cached
def test_func(farg,*args,**kwargs):
    time.sleep(2)
    return farg,args,kwargs

In [27]:
time_start = time.time()
test_func(1,2,3,a=4)
print(f'{time.time() - time_start} sec')
time_start = time.time()
test_func(1,2,3,a=4)
print(f'{time.time() - time_start} sec')

2.0033047199249268 sec
0.0009999275207519531 sec


#### 1.3 用lambda表达式和map函数编写一个程序，生成一个key为1到20之间整数，value为对应数的平方的字典(1和20都包含在内)。

In [17]:
# todo
lst = [x for x in range(1, 21)]
val = list(map(lambda x: x*x, lst))
dic = {x:y for x, y in zip(lst, val)}
print(dic)

{1: 1, 2: 4, 3: 9, 4: 16, 5: 25, 6: 36, 7: 49, 8: 64, 9: 81, 10: 100, 11: 121, 12: 144, 13: 169, 14: 196, 15: 225, 16: 256, 17: 289, 18: 324, 19: 361, 20: 400}


#### 1.4 递归函数
以方程：$x^3-0.2x^2-0.2x-1.2=0$为例，根据牛顿迭代法编写程序，求该方程的根，精度为0.00000001（该方程只有一个根）

提示：牛顿迭代法——设方程$f(x)=0$的根为$r$，取$x_0$作为$r$的初始近似值，过点($x_0$, $f(x_0)$)作$f(x)$的切线，与x轴交点的横坐标为$x_1=x_0-\frac{f(x_0)}{f'(x_0)}$，$x_1$称为$r$的一次近似值。再过点$(x_1,f(x_1)$作切线，继续迭代……
$x_{n+1}=x_n-\frac{f(x_n)}{f'(x_n)}$

In [18]:
# todo
def func(x):
    f = x**3 - 0.2*(x**2) - 0.2*x - 1.2
    f_ = 3*(x**2) - 0.4*x - 0.2
    x_ = x - f/f_
    if (abs(x-x_)<0.00000001):
        return x
    return func(x_)

sol = func(2)
print(sol)

1.2000000000545015


## 2 进阶题

#### 2.1 运行下面的代码，根据异常信息进行分析，定位出错误源头，并修复

In [19]:
from functools import reduce

def str2num(s):
    return float(s)

def calc(exp):
    ss = exp.split('+')
    ns = map(str2num, ss)
    return reduce(lambda acc, x: acc + x, ns)

def main():
    r = calc('100 + 200 + 345')
    print('100 + 200 + 345 =', r)
    r = calc('99 + 88 + 7.6')
    print('99 + 88 + 7.6 =', r)

main()

100 + 200 + 345 = 645.0
99 + 88 + 7.6 = 194.6


### 以下2.2与2.3的bonus question大家都可以尝试，如果两个尝试都正确*只算一个的分数*。

#### 2.2 废弃功能提醒装饰器
有些时候我们希望用户用新功能逐渐替代老功能，但是又希望保有兼容性，本题希望实现装饰器@deprecated达到这个功能。**注意，希望可以只在函数第一次被定义时抛出提示，之后不再抛出。**示例如下：

```python
@deprecated
def f(): 
    pass

>>> f()
Function deprecated, do not use.
>>> f()
>>> f()
```

In [20]:
import functools

def deprecate(func):
    # todo
    num = 0
    def inner():
        nonlocal num
        if num == 0:
            func()
            print("Function deprecated, do not use.")
            num += 1
    return inner
    pass

@deprecate
def f(): 
    pass

In [21]:
f()

Function deprecated, do not use.


In [22]:
f()

#### bonus 2.1.1（两个bonus只算一个分）
对废弃功能提醒装饰器进行如下的改进：
1. 同时打印本函数的名字
1. 接受自定义提醒语句

举例：
```python
@deprecate('use f1 instead')
def f(): 
    pass

>>> f()
Function f is deprecated, use f1 instead
>>> f()
>>> f()
```

In [1]:
import functools

def deprecate(msg):
    # todo
    def decorator(func):
        num = 0
        def inner(str=""):
            nonlocal num
            if num == 0:
                print(func.__name__,"is deprecated, "+msg)
                num += 1
            return func()
        return inner
    return decorator
    pass

@deprecate('use f1 instead')
def f(): 
    pass

In [2]:
f()

f is deprecated, use f1 instead


In [3]:
f()

#### 2.3 用字典表示图
本题中我们将用边表示的无向图构建一个字典表示的无向图，这种表示在图的遍历中很有效，最后我们选择性地可以尝试找到该图的两个节点之间的最短路径（提示：用BFS算法）。

举例：
```python
>>> edges = [["A", "B"], ["A", "E"], 
     ["A", "C"], ["B", "D"], 
     ["B", "E"], ["C", "F"], 
     ["C", "G"], ["D", "E"]]
>>> build_graph(edges)
{'A': ['B', 'E', 'C'], 
 'B': ['A', 'D', 'E'],
 'E': ['A', 'B', 'D'],
 'C': ['A', 'F', 'G'],
 'D': ['B', 'E'],
 'F': ['C'],
 'G': ['C']}

```

In [26]:
from collections import defaultdict

# 建图
def build_graph(edges):
    # todo
    dic = {}
    for edge in edges:
        if edge[0] not in dic:
            dic[edge[0]] = [edge[1]]
        else:
            dic[edge[0]] += edge[1]
        if edge[1] not in dic:
            dic[edge[1]] = [edge[0]]
        else:
            dic[edge[1]] += edge[0]
    return dic
    pass

edges = [
    ["A", "B"], ["A", "E"],
    ["A", "C"], ["B", "D"],
    ["B", "E"], ["C", "F"],
    ["C", "G"], ["D", "E"]
]

graph = build_graph(edges)
graph

{'A': ['B', 'E', 'C'],
 'B': ['A', 'D', 'E'],
 'E': ['A', 'B', 'D'],
 'C': ['A', 'F', 'G'],
 'D': ['B', 'E'],
 'F': ['C'],
 'G': ['C']}

#### bonus 2.3.1:(打印输出两节点间最短路径)，两个bonus只算一个分

In [27]:
#Find the shortest path in the graph using dictionaries

def BFS_SP(graph, start, goal):
    # todo
    point = [x for x in graph.keys()]
    q = [start]
    num = 0
    path = [start]
    while num < len(point):
        tmp = q[num]
        num += 1
        for x in graph[tmp[-1]]:
            if x not in q:
                q.append(tmp+x)
                if x == goal:
                    print("Shortest path = ", end='')
                    for i in tmp:
                        print(i, end=' ')
                    print(x)
                    num = len(point)
                    break
    pass

BFS_SP(graph, 'A', 'D')

Shortest path = A B D
